# Ingest Real-Time Stock Data to Iguazio NoSQL and Time-series DB
the following example function ingest real-time stock information from an internet service (Yahoo finance api) into iguazio platform.<br>
everytime the data is updated it updates a NoSQL table with the recent metadata and updates the time-series DB with the new metrics (price and volume)

The same code can run inside a nuclio (serverless) function and be automatically triggered on a predefined schedule (cron) or through HTTP requests<br>

the example demonstrate the use of `%nuclio` magic commands to specify environment variables, package dependencies,<br>configurations (such as the cron schedule), and to deploy functions automatically onto a cluster.

In [1]:
# nuclio: ignore
# if the nuclio-jupyter package is not installed run !pip install nuclio-jupyter
import nuclio 

copy the local credentials to the nuclio function config (-c option doesn't initialize locally)

In [2]:
%nuclio env -c V3IO_ACCESS_KEY=${V3IO_ACCESS_KEY}
%nuclio env -c V3IO_USERNAME=${V3IO_USERNAME}
%nuclio env -c V3IO_API=${V3IO_API}

### Set function configuration 
use a cron trigger with 5min interval and define the base image<br>
for more details check [nuclio function configuration reference](https://github.com/nuclio/nuclio/blob/master/docs/reference/function-configuration/function-configuration-reference.md)

In [4]:
%%nuclio config 
spec.triggers.secs.kind = "cron"
spec.triggers.secs.attributes.interval = "300s"
spec.build.baseImage = "python:3.6-jessie"

%nuclio: setting spec.triggers.secs.kind to 'cron'
%nuclio: setting spec.triggers.secs.attributes.interval to '300s'
%nuclio: setting spec.build.baseImage to 'python:3.6-jessie'


### Install required packages
`%nuclio cmd` allows you to run image build instructions and install packages<br>
Note: `-c` option will only install in nuclio, not locally

In [5]:
%%nuclio cmd -c 
pip install lxml
pip install yfinance
pip install requests
pip install v3io_frames

## Nuclio function implementation
this function can run in Jupyter or in nuclio (real-time serverless)

In [2]:
import json
import requests
import yfinance as yf
import os
import pandas as pd
import datetime
import v3io_frames as v3f

stocks = os.getenv('STOCK_LIST','GOOG,MSFT,AMZN,AAPL,INTC')
stock_syms = stocks.split(',')
client = v3f.Client('framesd:8081',container='bigdata')

# v3io update expression template 
expr_template = "symbol='{symbol}';price={price};volume={volume};last_updated='{last_updated}'"

last_trade_times = {}

# nuclio handler function 
def handler(context, event):
    
    stocks=[]; times=[]; volumes=[]; prices=[]
    
    for sym in stock_syms:
        last = last_trade_times.get(sym)
        
        hist = yf.Ticker(sym).history(period='60m', interval='60m')
        time = hist.index[len(hist) - 1]
        record = hist.loc[time]
        last = last_trade_times.get(sym)
        
        # update the stocks table and TSDB metrics in case of new data 
        if not last or time > last:
            
            # update NoSQL table with stock data
            stock = {'symbol': sym, 'price': record['Close'], 'volume': record['Volume'], 'last_updated': time}
            expr = expr_template.format(**stock)
            context.logger.debug_with('update expression', symbol=sym, expr=expr)
            client.execute('kv','stocks','update', args={'key':sym, 'expression': expr})
        
            # update time-series DB with price and volume metrics (use pandas dataframe with a single row, indexed by date)
            last_trade_times[sym] = time 
            stocks += [sym]
            times +=[time]
            volumes += [record['Volume']]
            prices += [record['Close']]
               
    # write price and volume metrics to the Time-Series DB, add exchange label
    if len(stocks)>0:
        df = pd.DataFrame({'volume':volumes,'price': prices}, index=[times,stocks], columns=['volume','price'])
        df.index.names=['time','symbol']
        context.logger.debug_with('writing data to TSDB', stocks=stocks)
        client.write(backend='tsdb', table='stock_metrics',dfs=df)
        
    return 'done'

## Function invocation
the following section simulates nuclio function invocation and will emit the function results

In [3]:
# nuclio: ignore
# create a test event and invoke the function locally 
event = nuclio.Event(body='')
handler(context, event)

'done'

## Deploy a function onto a cluster
the `%nuclio deploy` command deploy functions into a cluster, make sure the notebook is saved prior to running it !<br>check the help (`%nuclio help deploy`) for more information

In [4]:
%nuclio deploy -p stocks -n read-stocks

[nuclio] 2020-08-05 06:08:33,321 (info) Building processor image
[nuclio] 2020-08-05 06:08:35,344 (info) Build complete
[nuclio] 2020-08-05 06:08:39,389 (info) Function deploy complete
[nuclio] 2020-08-05 06:08:39,398 done creating read-stocks, function address: 192.168.224.185:32488
%nuclio: function deployed
